In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten , Dropout
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam

In [ ]:
conv_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(150,150,3)
)

In [ ]:
conv_base.summary()

In [ ]:
model  = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
conv_base.trainable  = False

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

# Specify the directory containing all the images
data_directory = r'/content/dogs_vs_cats/train'

# Split the data into training and validation datasets
training_ds = image_dataset_from_directory(
    directory=data_directory,
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(150, 150),
    shuffle=True,
    seed=42,
    validation_split=0.1,
    subset='training'
)

validation_ds = image_dataset_from_directory(
    directory=data_directory,
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(150, 150),
    shuffle=True,
    seed=42,
    validation_split=0.1,
    subset='validation'
)


In [ ]:
def Normalizaion(image,label):
    image  = tf.cast(image/255.,tf.float32)
    return image,label

training_ds = training_ds.map(Normalizaion)
validation_ds  = validation_ds.map(Normalizaion)

In [ ]:
# Learning rate finder variables
lrs = []
losses = []
min_lr = 1e-7  # Minimum learning rate
max_lr = 1     # Maximum learning rate
lr_factor = (max_lr / min_lr) ** (1 / len(training_ds))  # Exponential increment


In [ ]:
# Use a custom training loop to find the optimal learning rate
optimizer = Adam(learning_rate=min_lr)
loss_fn = keras.losses.BinaryCrossentropy()
train_loss_metric = keras.metrics.Mean()

for batch_idx, (images, labels) in enumerate(training_ds):
    # Update learning rate
    lr = min_lr * (lr_factor ** batch_idx)
    optimizer.learning_rate.assign(lr)
    lrs.append(lr)

    # Forward pass
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)

    # Backward pass and optimization
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss_metric.update_state(loss)

    # Record the loss
    losses.append(train_loss_metric.result().numpy())

    # Stop once we've covered enough steps
    if lr > max_lr:
        break


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Plot the learning rate finder results
plt.figure(figsize=(10, 6))
plt.plot(lrs, losses)
plt.xscale('log')
plt.xlabel('Learning Rate (Log Scale)')
plt.ylabel('Loss')
plt.title('Learning Rate Finder')
plt.show()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001) ,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(
    training_ds,
    epochs=30,
    validation_data=validation_ds
)

In [ ]:
model.save(r'C:\model_save\dog_cat\dog_cat_woa.h5')
print("Model saved successfully!")

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('Model Accuracy: Training vs Validation')
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.title('Model loss: Training vs Validation')
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load your trained model (replace the path with your model's file path)
model = tf.keras.models.load_model(r'C:\model_save\dog_cat\dog_cat_woa.h5')

# Function to preprocess the image and classify it using the custom model
def classify_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to model input size
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Predict using your trained model
    prediction = model.predict(img_array)  # Sigmoid returns a single probability value

    # Display the image
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

    # Print output based on the model's prediction
    if prediction[0][0] < 0.5:
        print("Predicted: Cat")
    else:
        print("Predicted: Dog")

# Folder containing images
image_folder = r'G:\image_dataset\valid'

# Loop through the images in the folder and classify each
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
        print(f"Classifying {img_name}...")
        classify_image(img_path)
        print("=" * 40)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the base directory containing your image data
data_directory = r'G:\image_dataset\cat_or_dog'

# Define the batch size
batch_size = 16

# Data augmentation for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1  # Split 20% for validation
)

# Normalization for validation data
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Flow from directory for training set (80% of the data)
train_generator = train_datagen.flow_from_directory(
    directory=data_directory,  # Directory for both training and validation data
    target_size=(150, 150),
    batch_size=batch_size,
    seed=42,
    class_mode='binary',  # Since it's binary classification (cat or dog)
    subset='training'  # Specifies this is the training part of the data
)

# Flow from directory for validation set (20% of the data)
validation_generator = train_datagen.flow_from_directory(
    directory=data_directory,  # Directory for both training and validation data
    target_size=(150, 150),
    batch_size=batch_size,
    seed=42,
    class_mode='binary',  # Since it's binary classification (cat or dog)
    subset='validation'  # Specifies this is the validation part of the data
)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001) ,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

In [ ]:
model.save(r'C:\model_save\dog_cat\dog_cat_wa.h5')
print("Model saved successfully!")

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('Model Accuracy: Training vs Validation')
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.title('Model loss: Training vs Validation')
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load your trained model (replace the path with your model's file path)
model = tf.keras.models.load_model(r'C:\model_save\dog_cat\dog_cat_wa.h5')

# Function to preprocess the image and classify it using the custom model
def classify_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to model input size
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Predict using your trained model
    prediction = model.predict(img_array)  # Sigmoid returns a single probability value

    # Display the image
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

    # Print output based on the model's prediction
    if prediction[0][0] < 0.5:
        print("Predicted: Cat")
    else:
        print("Predicted: Dog")

# Folder containing images
image_folder = r'G:\image_dataset\valid'

# Loop through the images in the folder and classify each
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
        print(f"Classifying {img_name}...")
        classify_image(img_path)
        print("=" * 40)


In [ ]:
model  = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001) ,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

In [ ]:
model.save(r'C:\model_save\dog_cat\dog_cat_wa_dropout.h5')
print("Model saved successfully!")

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('Model Accuracy: Training vs Validation')
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.title('Model loss: Training vs Validation')
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load your trained model (replace the path with your model's file path)
model = tf.keras.models.load_model(r'C:\model_save\dog_cat\dog_cat_wa_dropout.h5')

# Function to preprocess the image and classify it using the custom model
def classify_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to model input size
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Predict using your trained model
    prediction = model.predict(img_array)  # Sigmoid returns a single probability value

    # Display the image
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

    # Print output based on the model's prediction
    if prediction[0][0] < 0.5:
        print("Predicted: Cat")
    else:
        print("Predicted: Dog")

# Folder containing images
image_folder = r'G:\image_dataset\valid'

# Loop through the images in the folder and classify each
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
        print(f"Classifying {img_name}...")
        classify_image(img_path)
        print("=" * 40)


In [ ]:
# @title
import numpy as np
import matplotlib.pyplot as plt

# Simulating random learning rate vs. loss plots
x = np.logspace(-7, 0, 100)  # Learning rates (log scale)
y1 = 1 / (1 + np.exp(-3 * (np.log10(x) + 3))) + np.random.uniform(-0.05, 0.05, size=100)  # Sharp drop, plateau
y2 = 1 / (1 + np.exp(-2 * (np.log10(x) + 1))) + 0.1 * np.log10(x)  # Diverging loss
y3 = 1 / (1 + np.exp(-2 * (np.log10(x) + 2))) + np.random.uniform(-0.03, 0.03, size=100)  # Smooth loss curve

# Plotting the graphs
plt.figure(figsize=(15, 10))

# Plot 1: Optimal range found
plt.subplot(3, 1, 1)
plt.plot(x, y1, label="Loss vs. Learning Rate", color='blue')
plt.xscale('log')
plt.xlabel('Learning Rate (Log Scale)')
plt.ylabel('Loss')
plt.title('Learning Rate Finder - Sharp Drop, Plateau Found')
plt.axvline(x=1e-4, color='green', linestyle='--', label='Recommended LR')
plt.legend()

# Plot 2: Diverging loss
plt.subplot(3, 1, 2)
plt.plot(x, y2, label="Loss vs. Learning Rate", color='red')
plt.xscale('log')
plt.xlabel('Learning Rate (Log Scale)')
plt.ylabel('Loss')
plt.title('Learning Rate Finder - Diverging Loss at High LR')
plt.axvline(x=1e-3, color='orange', linestyle='--', label='Safe Lower LR')
plt.legend()

# Plot 3: Smooth curve with gradual improvement
plt.subplot(3, 1, 3)
plt.plot(x, y3, label="Loss vs. Learning Rate", color='purple')
plt.xscale('log')
plt.xlabel('Learning Rate (Log Scale)')
plt.ylabel('Loss')
plt.title('Learning Rate Finder - Smooth Gradual Improvement')
plt.axvline(x=3e-3, color='green', linestyle='--', label='Optimal LR')
plt.legend()

plt.tight_layout()
plt.show()
